__<span style="background-color:#ffffe0">第3章　時系列予測モデル構築・超入門</span>__

# 3.3　時系列の予測モデルを構築してみよう

## 3.3.7　時系列特徴量を生成しテーブルデータを作ろう！

* ラグ特徴量（ラグ1）
* ラグ特徴量（ラグ12）
* ローリング特徴量（1期前までの12ヶ月平均）
* エクスパンディング特徴量（1期前までの平均）
* トレンド特徴量（線形）

### 準備（必要なモジュールとデータの読み込み）

In [1]:
#
# 必要なモジュールの読み込み
#

import pandas as pd

import warnings
warnings.simplefilter('ignore')

In [2]:
#
# 必要なデータセット（時系列データ）の読み込み
#

dataset='AirPassengers.csv'  #データセットのファイル名
df=pd.read_csv(
    dataset,
    index_col='Month',   #変数「Month」をインデックスに設定
    parse_dates=True)   #インデックスを日付型に設定

### ラグ特徴量の生成

In [3]:
#
# ラグ特徴量（ラグ1）の生成
#

lag1 = df.shift(1)

print(lag1.head(15)) #確認

            Passengers
Month                 
1949-01-01         NaN
1949-02-01       112.0
1949-03-01       118.0
1949-04-01       132.0
1949-05-01       129.0
1949-06-01       121.0
1949-07-01       135.0
1949-08-01       148.0
1949-09-01       148.0
1949-10-01       136.0
1949-11-01       119.0
1949-12-01       104.0
1950-01-01       118.0
1950-02-01       115.0
1950-03-01       126.0


In [4]:
#
# ラグ特徴量（ラグ12）の生成
#

lag12 = df.shift(12)

print(lag12.head(15)) #確認

            Passengers
Month                 
1949-01-01         NaN
1949-02-01         NaN
1949-03-01         NaN
1949-04-01         NaN
1949-05-01         NaN
1949-06-01         NaN
1949-07-01         NaN
1949-08-01         NaN
1949-09-01         NaN
1949-10-01         NaN
1949-11-01         NaN
1949-12-01         NaN
1950-01-01       112.0
1950-02-01       118.0
1950-03-01       132.0


### ローリング特徴量（1期前までの12ヶ月平均）

In [5]:
#
# ローリング特徴量（1期前までの12ヶ月平均）
#

window12 = lag1.rolling(window=12).mean()

print(window12.head(15)) #確認

            Passengers
Month                 
1949-01-01         NaN
1949-02-01         NaN
1949-03-01         NaN
1949-04-01         NaN
1949-05-01         NaN
1949-06-01         NaN
1949-07-01         NaN
1949-08-01         NaN
1949-09-01         NaN
1949-10-01         NaN
1949-11-01         NaN
1949-12-01         NaN
1950-01-01  126.666667
1950-02-01  126.916667
1950-03-01  127.583333


### エクスパンディング特徴量（1期前までの平均）

In [6]:
#
# エクスパンディング特徴量（1期前までの平均）
#

expanding = lag1.expanding().mean()

print(expanding.head(15)) #確認

            Passengers
Month                 
1949-01-01         NaN
1949-02-01  112.000000
1949-03-01  115.000000
1949-04-01  120.666667
1949-05-01  122.750000
1949-06-01  122.400000
1949-07-01  124.500000
1949-08-01  127.857143
1949-09-01  130.375000
1949-10-01  131.000000
1949-11-01  129.800000
1949-12-01  127.454545
1950-01-01  126.666667
1950-02-01  125.769231
1950-03-01  125.785714


### 作成した時系列特徴量を結合しテーブルデータを生成

In [7]:
#
# 作成した時系列特徴量を結合しテーブルデータを生成
#

## データを結合
df_tbl = pd.concat([df,
                    lag1,
                    lag12,
                    window12,
                    expanding],
                   axis=1)

## 変数名を設定
df_tbl.columns = ['y',
                  'lag1',
                  'lag12',
                  'window12',
                  'expanding']

print(df_tbl.head(15)) #確認

              y   lag1  lag12    window12   expanding
Month                                                
1949-01-01  112    NaN    NaN         NaN         NaN
1949-02-01  118  112.0    NaN         NaN  112.000000
1949-03-01  132  118.0    NaN         NaN  115.000000
1949-04-01  129  132.0    NaN         NaN  120.666667
1949-05-01  121  129.0    NaN         NaN  122.750000
1949-06-01  135  121.0    NaN         NaN  122.400000
1949-07-01  148  135.0    NaN         NaN  124.500000
1949-08-01  148  148.0    NaN         NaN  127.857143
1949-09-01  136  148.0    NaN         NaN  130.375000
1949-10-01  119  136.0    NaN         NaN  131.000000
1949-11-01  104  119.0    NaN         NaN  129.800000
1949-12-01  118  104.0    NaN         NaN  127.454545
1950-01-01  115  118.0  112.0  126.666667  126.666667
1950-02-01  126  115.0  118.0  126.916667  125.769231
1950-03-01  141  126.0  132.0  127.583333  125.785714


In [8]:
# 欠測値削除
df_tbl = df_tbl.dropna()

print(df_tbl.head(15)) #確認

              y   lag1  lag12    window12   expanding
Month                                                
1950-01-01  115  118.0  112.0  126.666667  126.666667
1950-02-01  126  115.0  118.0  126.916667  125.769231
1950-03-01  141  126.0  132.0  127.583333  125.785714
1950-04-01  135  141.0  129.0  128.333333  126.800000
1950-05-01  125  135.0  121.0  128.833333  127.312500
1950-06-01  149  125.0  135.0  129.166667  127.176471
1950-07-01  170  149.0  148.0  130.333333  128.388889
1950-08-01  170  170.0  148.0  132.166667  130.578947
1950-09-01  158  170.0  136.0  134.000000  132.550000
1950-10-01  133  158.0  119.0  135.833333  133.761905
1950-11-01  114  133.0  104.0  137.000000  133.727273
1950-12-01  140  114.0  118.0  137.833333  132.869565
1951-01-01  145  140.0  115.0  139.666667  133.166667
1951-02-01  150  145.0  126.0  142.166667  133.640000
1951-03-01  178  150.0  141.0  144.166667  134.269231


### トレンド特徴量（線形）

In [9]:
#
# トレンド特徴量（線形）
#

df_tbl['t'] = pd.RangeIndex(start=0, 
                            stop=len(df_tbl))

print(df_tbl) #確認

              y   lag1  lag12    window12   expanding    t
Month                                                     
1950-01-01  115  118.0  112.0  126.666667  126.666667    0
1950-02-01  126  115.0  118.0  126.916667  125.769231    1
1950-03-01  141  126.0  132.0  127.583333  125.785714    2
1950-04-01  135  141.0  129.0  128.333333  126.800000    3
1950-05-01  125  135.0  121.0  128.833333  127.312500    4
...         ...    ...    ...         ...         ...  ...
1960-08-01  606  622.0  559.0  459.416667  273.136691  127
1960-09-01  508  606.0  463.0  463.333333  275.514286  128
1960-10-01  461  508.0  407.0  467.083333  277.163121  129
1960-11-01  390  461.0  362.0  471.583333  278.457746  130
1960-12-01  432  390.0  405.0  473.916667  279.237762  131

[132 rows x 6 columns]


### CSVファイルとして出力

In [10]:
df_tbl.to_csv('df_tbl.csv' )